In [ ]:
pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import wikipedia
import re
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Download the Wikipedia data
wikipedia.set_lang('en')
wikipedia_data = []
for title in wikipedia.random(5000):
    try:
        page = wikipedia.page(title)

        text = re.sub(r'\s+', ' ', page.content)
        wikipedia_data.append(text)
    except:
        pass


/usr/local/lib/python3.9/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.9/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
wikipedia_data

['Gonatodes ligiae is a species of lizard in the Sphaerodactylidae family found in Venezuela. == References ==',
 'Paisean Faisean ("Fashion Passion") is an Irish language television dating show broadcast on TG4. It was presented in the first three series by Aoife Ní Thuairisg, and in the fourth series by Bláthnaid Nic Dhonnchadha and Mairéad Ní Chuaig. The fifth series saw Nic Dhonnchadha present alongside Síle Seoige. == Format == Magma films bought the format from Digital Rights Group with the line "Boys buy clothes for girls" from that Clodagh Freeman created the series. Magma films get a percentage of international rights as they bought the format from paper and brought it to the screen first. The format of the show traditionally involves three men choosing outfits for one woman – although the episode aired on 19 November 2007 featured four gay men. The suitors are shown a short pre-recorded clip of the girl, wearing jeans and a white T-shirt, in which she introduces herself and g

In [ ]:
# open file
with open('/content/drive/MyDrive/Colab Notebooks/wikipedia_data.txt', 'w') as f: 
  # write elements of list
    for items in wikipedia_data:
        f.write('%s\n' %items)
  
# close the file
f.close()

In [ ]:
# wikipedia_data = []
# # open file in read mode
# f = open('/content/drive/MyDrive/Colab Notebooks/wikipedia_data.txt', 'r')
 
# # display content of the file
# wikipedia_data = f.read()
 
# # close the file
# f.close()

In [ ]:
wikipedia_data = list(wikipedia_data)


In [ ]:
# Define a function to clean the text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

# Define a function to tokenize the text
def tokenize_text(text):
    return text.split()

In [ ]:
# Implement the TF-IDF vectorizer
vectorizer = TfidfVectorizer(preprocessor=clean_text, tokenizer=tokenize_text)
tf_idf_matrix = vectorizer.fit_transform(wikipedia_data)
feature_names = vectorizer.vocabulary_

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# Define a function to calculate Jaccard coefficient
def jaccard_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0

In [ ]:
# fountion to search query in data corpus
def search(query, k=10):
    # Tokenize the query text into words
    query_tokens = set(tokenize_text(clean_text(query)))
    # Jaccard coefficient between the query text and all documents
    similarities = []
    for i in range(tf_idf_matrix.shape[0]):
        document_tokens = set(tokenize_text(wikipedia_data[i]))
        jaccard = jaccard_coefficient(query_tokens, document_tokens)
        if jaccard == 0:
            similarity = 0
        else:
            # Calculate the TF-IDF vector for the document text
            document_vector = tf_idf_matrix[i].toarray()[0]
            # Calculate the TF-IDF vector for the query text
            query_vector = np.zeros(len(feature_names))
            for token in query_tokens:
                if token in feature_names:
                    index = feature_names.get(token)
                    query_vector[index] += 1
            query_vector = query_vector / len(query_tokens)
            # Calculate the similarity between the query text and the document text
            similarity = np.dot(query_vector, document_vector) / np.linalg.norm(query_vector) / np.linalg.norm(document_vector) * jaccard
        similarities.append(similarity)
    # Return the top-k most similar documents
    top_k_indices = np.argsort(similarities)[::-1][:k]
    top_k_scores = [similarities[i] for i in top_k_indices]
    top_k_documents = [wikipedia_data[i] for i in top_k_indices]
    return top_k_documents, top_k_scores

In [ ]:
wikipedia_data

['Gonatodes ligiae is a species of lizard in the Sphaerodactylidae family found in Venezuela. == References ==',
 'Paisean Faisean ("Fashion Passion") is an Irish language television dating show broadcast on TG4. It was presented in the first three series by Aoife Ní Thuairisg, and in the fourth series by Bláthnaid Nic Dhonnchadha and Mairéad Ní Chuaig. The fifth series saw Nic Dhonnchadha present alongside Síle Seoige. == Format == Magma films bought the format from Digital Rights Group with the line "Boys buy clothes for girls" from that Clodagh Freeman created the series. Magma films get a percentage of international rights as they bought the format from paper and brought it to the screen first. The format of the show traditionally involves three men choosing outfits for one woman – although the episode aired on 19 November 2007 featured four gay men. The suitors are shown a short pre-recorded clip of the girl, wearing jeans and a white T-shirt, in which she introduces herself and g

In [ ]:
results, scores = search("television")
for i in range(len(results)):
    print(f"({i+1}) {results[i][:1000]}... (score: {scores[i]:.8f})")

(1) Radio G.R.E.M. is an Italian television series. == See also == List of Italian television series == References ==... (score: 0.03486414)
(2) GiantsVision was a pay-per-view television service for Major League Baseball games featuring the San Francisco Giants. GiantsVision was in operation for four seasons (1986–1989). Prior to this, the team's only local television outlet was KTVU, which had been broadcasting Giants games since 1961. == Background == GiantsVision was managed by San Jose-based Bay Area Interconnect. It was headed by Robert Hartzell and Mike Shapiro. Subscription costs began at $5.95 per game, with a full 45-game package available for $154. GiantsVision garnered an average viewership of 22,000 per game through the first two months of the 1988 season.GiantsVision was shut down at the conclusion of the 1989 season when it was purchased by SportsChannel and floated as SportsChannel Bay Area after the merger. By 1990, at least 105 San Francisco Giants games had been tele